# Winnowing 算法详解与实现

## 1. Winnowing 算法简介

Winnowing 是一种用于文档指纹提取的算法，常用于检测文档相似性和抄袭。它通过选择文档中 n-grams 的一个子集（称为指纹）来工作。选择过程基于哈希值，确保相似的文档会产生许多相同的指纹。

该算法的关键思想是：
- **局部性保留哈希 (Locality Preserving Hashing, LPH)**：相似的输入会产生相似的哈希值。Winnowing 通常不直接使用 LPH，而是依赖于对 n-grams 进行哈希处理。
- **窗口化 (Windowing)**：算法考虑一个固定大小的窗口内的哈希值。
- **选择最小哈希值**：在每个窗口中，选择具有最小哈希值的 n-gram（或其哈希值）作为指纹。为了避免选择过多的指纹或在哈希冲突时做出选择，通常会采用更健壮的策略，例如选择窗口中最右边的最小哈希值。

Winnowing 算法对于检测文档中的小段重复内容特别有效。

### Winnowing 算法的参数：
- **k (或 n)**：n-gram 的大小。这是用于生成文档基本单元的连续字符（或词语）的数量。
- **w (窗口大小)**：用于选择指纹的窗口大小。算法会查看 w 个连续的 n-gram 哈希值，并从中选择一个指纹。

### 算法步骤：
1. **文本预处理**：去除标点符号、转换为小写、去除停用词等（根据需求）。
2. **生成 k-grams**：将文档分割成 k 个连续字符（或词语）的序列。
3. **哈希 k-grams**：为每个 k-gram 计算一个哈希值。
4. **选择指纹**：
    a. 将 k-gram 的哈希值序列组织成窗口，每个窗口包含 `w` 个连续的哈希值。
    b. 在每个窗口中，选择最小的哈希值作为指纹。如果存在多个相同的最小哈希值，则选择最右边（或最左边，取决于实现策略）的那个，以确保一致性并减少指纹数量。
    c. 记录选定指纹及其在原始文档中的位置（可选，但对于定位相似部分很有用）。
5. **比较指纹集**：通过比较两个文档的指纹集来确定它们的相似度。常用的度量是 Jaccard 相似系数。

Winnowing 算法在保持检测精度的同时，能够显著减少需要存储和比较的数据量。

## 2. 安装和导入库

在此部分，我们将导入实现 Winnowing 算法及其测试所需的 Python 库。

In [1]:
import hashlib
import re
import time
import random
import string
import matplotlib.pyplot as plt
import numpy as np
import jieba # 确保jieba已导入

# 用于支持中文显示
plt.rcParams['font.sans-serif'] = ['SimHei']  # 指定默认字体
plt.rcParams['axes.unicode_minus'] = False  # 解决保存图像是负号'-'显示为方块的问题

print("库导入完成。")

库导入完成。


## 3. Winnowing 核心算法实现

我们将从头开始实现 Winnowing 算法。

In [2]:
def preprocess_text(text):
    """
    预处理文本：转换为小写，移除特定标点符号，标准化空格。
    保留必要的标点符号以便后续分词。
    """
    text = text.lower()
    # 移除大部分标点，但保留一些可能对分词有用的（例如连字符）
    # 或者更简单地，在分词后再处理标点
    # text = re.sub(r"[^\w\s\u4e00-\u9fff-]", "", text) # 示例：保留连字符
    text = re.sub(r"\s+", " ", text).strip()
    return text

def get_word_k_grams(text, k):
    """
    从文本生成词级别的 k-grams。
    对于中文，使用jieba分词。
    对于英文，按空格分词。
    k-gram 中的词用下划线 "_" 连接。
    """
    processed_text = preprocess_text(text)
    
    words = []
    # 判断是否包含中文字符
    is_chinese = any('\u4e00' <= char <= '\u9fff' for char in processed_text)
    
    if is_chinese:
        words = [word for word in jieba.cut(processed_text) if word.strip() and not re.fullmatch(r'[\W_]+', word)] # 过滤空词和纯标点
    else:
        # 英文分词，并移除标点
        # 先移除标点，再分词
        text_no_punct = re.sub(r'[^\w\s]', '', processed_text) 
        words = [word for word in text_no_punct.split() if word.strip()]

    if not words or len(words) < k:
        if words: # 如果有词但不够k个，可以将它们合并成一个k-gram
            return ["_".join(words)]
        return []
        
    k_grams = []
    for i in range(len(words) - k + 1):
        k_grams.append("_".join(words[i:i+k]))
    return k_grams

def hash_k_gram(k_gram, algorithm='md5'):
    """
    对单个 k-gram 进行哈希处理。
    使用 hashlib 生成哈希值，并取整数形式。
    """
    if algorithm == 'md5':
        return int(hashlib.md5(k_gram.encode('utf-8')).hexdigest(), 16)
    elif algorithm == 'sha1':
        return int(hashlib.sha1(k_gram.encode('utf-8')).hexdigest(), 16)
    # 可以添加其他哈希算法
    else:
        # 一个简单的非加密哈希函数作为备选，但建议使用标准库中的
        hash_val = 0
        for char in k_gram:
            hash_val = (hash_val * 31 + ord(char)) & 0xFFFFFFFF # 限制在32位
        return hash_val


def winnowing(text, k, w, hash_algorithm='md5'):
    """
    实现 Winnowing 算法，使用词级别的 k-grams。

    参数:
    text (str): 输入文本。
    k (int): 词 k-gram 的大小。
    w (int): 窗口大小。
    hash_algorithm (str): 用于哈希 k-gram 的算法。

    返回:
    list: 选定的指纹列表，每个元素是 (hash_value, position) 的元组。
          position 是词 k-gram 在词列表中的起始索引。
    """
    if not text or k <= 0 or w <= 0:
        return []

    # 1. 生成词级别的 k-grams (预处理已在 get_word_k_grams 内部完成)
    # 注意：这里的 k_grams_list 是由词（或词序列）组成的，而不是字符序列
    word_k_grams_list = get_word_k_grams(text, k)
    
    if not word_k_grams_list:
        return []

    # 2. 哈希 k-grams
    # 我们存储 (hash_value, original_position_in_word_list)
    # original_position 是 k-gram 在分词后的词列表中的起始索引
    hashed_k_grams = []
    for i, k_gram_str in enumerate(word_k_grams_list):
        hashed_k_grams.append((hash_k_gram(k_gram_str, hash_algorithm), i))

    if not hashed_k_grams:
        return []
        
    if len(hashed_k_grams) < w:
        # 如果哈希后的 k-gram 数量少于窗口大小，
        # 选择所有哈希中的最小者（最右边的）
        min_hash_val = float('inf')
        selected_fingerprint = None
        # 从右向左遍历以选择最右边的最小哈希
        for i in range(len(hashed_k_grams) - 1, -1, -1):
            h_val, pos = hashed_k_grams[i]
            if h_val <= min_hash_val:
                min_hash_val = h_val
                selected_fingerprint = (h_val, pos)
        return [selected_fingerprint] if selected_fingerprint else []

    # 3. 选择指纹
    fingerprints = {} # 使用字典来存储指纹，键为位置，值为哈希值

    for i in range(len(hashed_k_grams) - w + 1):
        window = hashed_k_grams[i : i + w] 
        min_h_in_window = float('inf')
        selected_pos_in_window = -1

        for j in range(w - 1, -1, -1):
            current_hash, current_pos = window[j]
            if current_hash <= min_h_in_window:
                min_h_in_window = current_hash
                selected_pos_in_window = current_pos
        
        fingerprints[selected_pos_in_window] = min_h_in_window

    sorted_fingerprints = sorted([(h, p) for p, h in fingerprints.items()], key=lambda x: x[1])

    return sorted_fingerprints

def jaccard_similarity(set1, set2):
    """
    计算两个集合的 Jaccard 相似系数。
    J(A,B) = |A ∩ B| / |A ∪ B|
    """
    intersection_size = len(set1.intersection(set2))
    union_size = len(set1.union(set2))
    if union_size == 0:
        return 1.0 if intersection_size == 0 else 0.0 # 两个空集的相似度为1，或根据定义为0
    return intersection_size / union_size

def compare_documents_winnowing(doc1_fingerprints, doc2_fingerprints):
    """
    比较两个文档的 Winnowing 指纹集并计算 Jaccard 相似度。
    指纹被视为哈希值本身。
    """
    # 提取指纹中的哈希值部分进行比较
    fingerprints1_hashes = {fp[0] for fp in doc1_fingerprints}
    fingerprints2_hashes = {fp[0] for fp in doc2_fingerprints}

    return jaccard_similarity(fingerprints1_hashes, fingerprints2_hashes)

print("Winnowing 核心算法函数定义完成。")

Winnowing 核心算法函数定义完成。


## 4. 测试和示例

现在我们将测试上面实现的 Winnowing 算法。

### 4.1 简单示例文本测试

In [5]:
# 测试参数
k_val = 5  # k-gram 大小
w_val = 4  # 窗口大小 (t-k+1, 所以如果 t=8, k=5, 那么 w = 8-5+1 = 4)
           # 这里的 w 是指连续 k-gram 哈希值的数量

text1 = "A do run run run, a do run run."
text2 = "A do run run, a do run run run." # 稍作修改
text3 = "The quick brown fox jumps over the lazy dog."
text4 = "The quick brown fox jumps over the lazy cat." # 与text3相似
text5 = "这是一个用于测试Winnowing算法的示例文本。"
text6 = "这是一个用于测试Winnowing算法的示例文本，增加一些内容。"
text7 = "完全不同的中文句子。"

# 预处理和生成指纹
fp1 = winnowing(text1, k_val, w_val)
fp2 = winnowing(text2, k_val, w_val)
fp3 = winnowing(text3, k_val, w_val)
fp4 = winnowing(text4, k_val, w_val)
fp5 = winnowing(text5, k_val, w_val)
fp6 = winnowing(text6, k_val, w_val)
fp7 = winnowing(text7, k_val, w_val)

print(f"文本 1: \"{text1}\"")
print(f"处理后: \"{preprocess_text(text1)}\"")
print(f"K-grams (k={k_val}): {get_word_k_grams(text1, k_val)}")
print(f"指纹 (k={k_val}, w={w_val}): {fp1}")
print("-" * 30)

print(f"文本 2: \"{text2}\"")
print(f"处理后: \"{preprocess_text(text2)}\"")
print(f"指纹 (k={k_val}, w={w_val}): {fp2}")
print("-" * 30)

print(f"文本 3: \"{text3}\"")
print(f"处理后: \"{preprocess_text(text3)}\"")
print(f"指纹 (k={k_val}, w={w_val}): {fp3}")
print("-" * 30)

print(f"文本 4: \"{text4}\"")
print(f"处理后: \"{preprocess_text(text4)}\"")
print(f"指纹 (k={k_val}, w={w_val}): {fp4}")
print("-" * 30)

print(f"文本 5: \"{text5}\"")
print(f"处理后: \"{preprocess_text(text5)}\"")
print(f"指纹 (k={k_val}, w={w_val}): {fp5}")
print("-" * 30)

print(f"文本 6: \"{text6}\"")
print(f"处理后: \"{preprocess_text(text6)}\"")
print(f"指纹 (k={k_val}, w={w_val}): {fp6}")
print("-" * 30)

print(f"文本 7: \"{text7}\"")
print(f"处理后: \"{preprocess_text(text7)}\"")
print(f"指纹 (k={k_val}, w={w_val}): {fp7}")
print("-" * 30)


# 计算相似度
similarity_1_2 = compare_documents_winnowing(fp1, fp2)
similarity_3_4 = compare_documents_winnowing(fp3, fp4)
similarity_1_3 = compare_documents_winnowing(fp1, fp3)
similarity_5_6 = compare_documents_winnowing(fp5, fp6)
similarity_5_7 = compare_documents_winnowing(fp5, fp7)


print(f"文本 1 和文本 2 的相似度: {similarity_1_2:.4f}")
print(f"文本 3 和文本 4 的相似度: {similarity_3_4:.4f}")
print(f"文本 1 和文本 3 的相似度 (预期很低): {similarity_1_3:.4f}")
print(f"文本 5 和文本 6 的相似度 (中文): {similarity_5_6:.4f}")
print(f"文本 5 和文本 7 的相似度 (中文，完全不同): {similarity_5_7:.4f}")

# 调试一个特殊情况：文本太短
short_text = "abc"
fp_short = winnowing(short_text, k=4, w=2) # k > len(text)
print(f"\n短文本: \"{short_text}\", k=4, w=2")
print(f"处理后: \"{preprocess_text(short_text)}\"")
print(f"指纹: {fp_short}")

shorter_text = "abcdef" # len = 6, k=5, w=4. num_kgrams = 2. len(hashed_k_grams) < w
fp_shorter = winnowing(shorter_text, k=5, w=4)
print(f"\n较短文本: \"{shorter_text}\", k=5, w=4")
print(f"处理后: \"{preprocess_text(shorter_text)}\"")
print(f"K-grams: {get_word_k_grams(shorter_text, 5)}")
hashed_k_grams = [(h, p) for h, p in winnowing(shorter_text, 5, 4)]
print(f"哈希后的K-grams: {hashed_k_grams}")  # 模拟内部hashed_k_grams
print(f"指纹: {fp_shorter}")
# 预期：由于 hashed_k_grams 长度 (2) < w (4)，会选择全局最小的那个哈希值。

# 验证窗口选择逻辑
# 文本 = "abcdefghij", k=3, w=4
# k-grams: abc, bcd, cde, def, efg, fgh, ghi, hij (8个k-grams)
# 哈希值: h(abc), h(bcd), ..., h(hij)
# 窗口 1: [h(abc), h(bcd), h(cde), h(def)] -> 选择最右边的最小值
# 窗口 2: [h(bcd), h(cde), h(def), h(efg)] -> 选择最右边的最小值
# ...
# 窗口 5: [h(efg), h(fgh), h(ghi), h(hij)] -> 选择最右边的最小值

debug_text = "abracadabra"
debug_k = 3
debug_w = 4
debug_fp = winnowing(debug_text, debug_k, debug_w)
print(f"\n调试文本: \"{debug_text}\", k={debug_k}, w={debug_w}")
print(f"处理后: \"{preprocess_text(debug_text)}\"")
kgrams_debug = get_word_k_grams(debug_text, debug_k)
print(f"K-grams: {kgrams_debug}")
hashed_kgrams_debug_vals = [hash_k_gram(kg) for kg in kgrams_debug]
print(f"哈希后的k-gram值 (前几位十六进制): {[hex(h)[:8] for h in hashed_kgrams_debug_vals]}")
print(f"指纹: {debug_fp}")
print(f"指纹哈希值 (前几位十六进制): {[hex(h[0])[:8] for h in debug_fp]}")

# 检查指纹是否来自原始哈希列表
original_hashes_with_pos_debug = []
word_k_grams_for_debug = get_word_k_grams(debug_text, debug_k)
for i, k_gram_str in enumerate(word_k_grams_for_debug):
    original_hashes_with_pos_debug.append((hash_k_gram(k_gram_str), i))

print("调试文本的原始(哈希值, 位置)对:")
for h, p in original_hashes_with_pos_debug:
    print(f"  位置 {p}: {word_k_grams_for_debug[p]} -> {hex(h)[:8]}")

print("调试文本的选定指纹 (哈希值, 位置):")
for h_fp, p_fp in debug_fp:
    print(f"  位置 {p_fp}: {word_k_grams_for_debug[p_fp]} -> {hex(h_fp)[:8]}")

# 验证选择最右边最小哈希的逻辑
# 假设哈希序列是 [ (10,0), (5,1), (8,2), (5,3), (12,4) ] 且 w = 3
# 窗口 1: [(10,0), (5,1), (8,2)] -> 最小值是 (5,1)。选择 (5,1)
# 窗口 2: [(5,1), (8,2), (5,3)] -> 最小值有 (5,1) 和 (5,3)。选择最右边的 (5,3)
# 窗口 3: [(8,2), (5,3), (12,4)]-> 最小值是 (5,3)。选择 (5,3)
# 指纹: {(5,1), (5,3)}

# 手动模拟 winnowing 内部逻辑
def manual_winnowing_select(hashed_k_grams_with_pos, w):
    fingerprints = {}
    if len(hashed_k_grams_with_pos) < w :
        if not hashed_k_grams_with_pos: return []
        min_h_val = float('inf')
        sel_pos = -1
        for i in range(len(hashed_k_grams_with_pos)-1, -1, -1):
            h_val, pos = hashed_k_grams_with_pos[i]
            if h_val <= min_h_val:
                min_h_val = h_val
                sel_pos = pos
        return [(min_h_val, sel_pos)] if sel_pos != -1 else []

    for i in range(len(hashed_k_grams_with_pos) - w + 1):
        window = hashed_k_grams_with_pos[i : i + w]
        min_h_in_window = float('inf')
        selected_pos_in_window = -1
        for j in range(w - 1, -1, -1): # 从右到左迭代
            current_hash, current_pos = window[j]
            if current_hash <= min_h_in_window:
                min_h_in_window = current_hash
                selected_pos_in_window = current_pos
        fingerprints[selected_pos_in_window] = min_h_in_window
    return sorted([(h, p) for p, h in fingerprints.items()], key=lambda x: x[1])

# 测试手动选择
test_hash_seq = [(10,0), (5,1), (8,2), (5,3), (12,4)]
manual_selected_fp = manual_winnowing_select(test_hash_seq, 3)
print(f"\n{test_hash_seq}, w=3 的手动选择测试结果: {manual_selected_fp}")
# 预期: [(5,1), (5,3)] (按位置排序)

test_hash_seq_2 = [(10,0), (10,1), (8,2), (9,3)]
manual_selected_fp_2 = manual_winnowing_select(test_hash_seq_2, 3)
# 窗口 1: [(10,0), (10,1), (8,2)] -> 最小值是 (8,2)。选择 (8,2)
# 窗口 2: [(10,1), (8,2), (9,3)] -> 最小值是 (8,2)。选择 (8,2)
# 指纹: {(8,2)}
print(f"{test_hash_seq_2}, w=3 的手动选择测试结果: {manual_selected_fp_2}")
# 预期: [(8,2)]

# 测试特定哈希序列对应的实际文本
# 需要找到能产生特定哈希顺序的k-gram。这很难。
# 相反，如果manual_winnowing_select按预期工作，可以信任其逻辑。

Text 1: "A do run run run, a do run run."
Processed: "a do run run run a do run run"
K-grams (k=5): ['a do ', ' do r', 'do ru', 'o run', ' run ', 'run r', 'un ru', 'n run', ' run ', 'run r', 'un ru', 'n run', ' run ', 'run a', 'un a ', 'n a d', ' a do', 'a do ', ' do r', 'do ru', 'o run', ' run ', 'run r', 'un ru', 'n run']
Fingerprints (k=5, w=4): [(75578398313496731125539448415744811306, 3), (16554977725856530988832416307719916510, 7), (16554977725856530988832416307719916510, 11), (116723463916231355165740518890253991284, 13), (151057954142194230245704733847123552673, 15), (159489971866563299873662894940114379945, 18), (75578398313496731125539448415744811306, 20), (16554977725856530988832416307719916510, 24)]
------------------------------
Text 2: "A do run run, a do run run run."
Processed: "a do run run a do run run run"
Fingerprints (k=5, w=4): [(75578398313496731125539448415744811306, 3), (16554977725856530988832416307719916510, 7), (116723463916231355165740518890253991284, 9), (

### 4.2 不同 k 和 w 值对结果的影响

我们将尝试不同的 k (k-gram 大小) 和 w (窗口大小) 值，观察它们如何影响指纹的数量和文档相似性检测。

- **较大的 k**：
    - 优点：更能抵抗微小的文本变化（例如单个字符的更改），因为更改一个字符只会影响 k 个 k-gram。指纹更具特异性。
    - 缺点：如果 k 太大，可能无法捕捉到较短的相似片段。产生的 k-gram 总数较少。
- **较小的 k**：
    - 优点：对文本更敏感，能捕捉更细微的相似性。
    - 缺点：可能产生大量指纹，对微小差异（如拼写错误）更敏感，可能导致相似文本的指纹差异较大。
- **较大的 w** (窗口大小，对应于较小的保证阈值 t = w + k - 1 的倒数关系，或者说，w 越大，指纹越稀疏):
    - 优点：产生的指纹更少，存储和比较更快。
    - 缺点：可能会漏掉一些较短的匹配片段，因为选择标准更严格（需要在更大的窗口中成为最小值）。
- **较小的 w** (窗口大小，对应于较大的保证阈值 t 的倒数关系，或者说，w 越小，指纹越密集):
    - 优点：能捕捉到更短的匹配片段，指纹更密集。
    - 缺点：产生更多指纹，计算和存储开销更大。

选择合适的 k 和 w 取决于具体的应用场景和文本特性。通常需要实验来找到最优组合。
一个常见的选择是 k=5 到 k=10，w 也是一个类似范围内的值，或者根据保证阈值 t 来确定 (例如，如果希望保证检测到长度至少为 t 的匹配，则 w = t - k + 1)。

In [7]:
texts_for_param_test = [
    "The quick brown fox jumps over the lazy dog near the bank of the river.",
    "A quick brown fox also jumps over the lazy dog near the bank of the river.", # 轻微修改
    "The lazy dog is jumped over by a quick brown fox near the river bank.", # 重新排序但相似
    "Completely different sentence that has no relation to the others." # 完全不同的句子
]

k_values = [3, 5, 7]
w_values = [2, 4, 6] # w 必须 >= 1

results_param_test = []

print("运行参数测试 (k 和 w 值):")
for i, text1_content in enumerate(texts_for_param_test):
    for j, text2_content in enumerate(texts_for_param_test):
        if i >= j: # 避免重复对比和自我比较
            continue
        print(f"\n比较文本 {i+1} 和文本 {j+1}:")
        print(f"  文本 {i+1}: \"{text1_content[:50]}...\"")
        print(f"  文本 {j+1}: \"{text2_content[:50]}...\"")
        for k_val_pt in k_values:
            for w_val_pt in w_values:
                if w_val_pt == 0 : continue # w 必须 > 0
                # 确保 k 对于预处理后可能变短的文本不会过大
                # 对于这个一般测试，假设预处理后的文本长度足够使用选定的 k 值

                fp_text1 = winnowing(text1_content, k_val_pt, w_val_pt)
                fp_text2 = winnowing(text2_content, k_val_pt, w_val_pt)

                num_fp1 = len(fp_text1)
                num_fp2 = len(fp_text2)
                similarity = compare_documents_winnowing(fp_text1, fp_text2)

                results_param_test.append({
                    "text_pair": (i+1, j+1),
                    "k": k_val_pt,
                    "w": w_val_pt,
                    "num_fp1": num_fp1,
                    "num_fp2": num_fp2,
                    "similarity": similarity
                })
                print(f"    k={k_val_pt}, w={w_val_pt}: 相似度={similarity:.3f} (指纹1数量:{num_fp1}, 指纹2数量:{num_fp2})")

# 可以将 results_param_test 转换为 Pandas DataFrame 进行更详细的分析
# import pandas as pd
# df_results = pd.DataFrame(results_param_test)
# print("\n参数测试结果 (DataFrame):")
# print(df_results)

运行参数测试 (k 和 w 值):

比较文本 1 和文本 2:
  文本 1: "The quick brown fox jumps over the lazy dog near t..."
  文本 2: "A quick brown fox also jumps over the lazy dog nea..."
    k=3, w=2: 相似度=0.857 (指纹1数量:45, 指纹2数量:46)
    k=3, w=4: 相似度=0.808 (指纹1数量:27, 指纹2数量:27)
    k=3, w=6: 相似度=0.833 (指纹1数量:20, 指纹2数量:20)
    k=5, w=2: 相似度=0.761 (指纹1数量:40, 指纹2数量:43)
    k=5, w=4: 相似度=0.733 (指纹1数量:25, 指纹2数量:27)
    k=5, w=6: 相似度=0.750 (指纹1数量:16, 指纹2数量:19)
    k=7, w=2: 相似度=0.706 (指纹1数量:42, 指纹2数量:45)
    k=7, w=4: 相似度=0.733 (指纹1数量:25, 指纹2数量:27)
    k=7, w=6: 相似度=0.818 (指纹1数量:19, 指纹2数量:21)

比较文本 1 和文本 3:
  文本 1: "The quick brown fox jumps over the lazy dog near t..."
  文本 3: "The lazy dog is jumped over by a quick brown fox n..."
    k=3, w=2: 相似度=0.583 (指纹1数量:45, 指纹2数量:41)
    k=3, w=4: 相似度=0.438 (指纹1数量:27, 指纹2数量:25)
    k=3, w=6: 相似度=0.476 (指纹1数量:20, 指纹2数量:17)
    k=5, w=2: 相似度=0.397 (指纹1数量:40, 指纹2数量:43)
    k=5, w=4: 相似度=0.382 (指纹1数量:25, 指纹2数量:22)
    k=5, w=6: 相似度=0.391 (指纹1数量:16, 指纹2数量:16)
    k=7, w=2: 相似度=0.2

### 4.3 文本变化对相似度的影响实验 (类似 SimHash 和 MinHash)

我们将创建一个原始文本，然后对其进行不同程度的修改，观察 Winnowing 算法计算出的相似度如何变化。

修改类型：
1.  **字符替换**：随机替换文本中的一些字符。
2.  **字符插入**：随机在文本中插入一些字符。
3.  **字符删除**：随机删除文本中的一些字符。
4.  **段落重排/句子重排**（对于 Winnowing，这可能影响不大，因为它主要关注局部 k-gram 匹配，但如果重排导致 k-gram 序列变化，则会有影响）。Winnowing 对局部顺序敏感。

我们将使用固定的 k 和 w 值进行此实验。

In [ ]:
def generate_random_string(length):
    return ''.join(random.choice(string.ascii_lowercase + string.digits + ' ') for _ in range(length))

def modify_text(original_text, change_percentage, modification_type="replace"):
    """
    修改文本以测试相似度算法。
    change_percentage: 0.0 to 1.0
    modification_type: "replace", "insert", "delete"
    """
    text_list = list(original_text)
    num_changes = int(len(text_list) * change_percentage)
    modified_text_list = list(text_list) # Make a copy

    if modification_type == "replace":
        indices_to_change = random.sample(range(len(text_list)), min(num_changes, len(text_list)))
        for idx in indices_to_change:
            modified_text_list[idx] = random.choice(string.ascii_lowercase + ' ') # Replace with a random char
        return "".join(modified_text_list)

    elif modification_type == "insert":
        for _ in range(num_changes):
            insert_idx = random.randint(0, len(modified_text_list))
            char_to_insert = random.choice(string.ascii_lowercase + ' ')
            modified_text_list.insert(insert_idx, char_to_insert)
        return "".join(modified_text_list)

    elif modification_type == "delete":
        if not modified_text_list: return ""
        indices_to_delete = sorted(random.sample(range(len(modified_text_list)), min(num_changes, len(modified_text_list))), reverse=True)
        for idx in indices_to_delete:
            del modified_text_list[idx]
        return "".join(modified_text_list)

    return original_text # Default if type is unknown


# --- 实验参数 ---
base_text_variation = """Winnie the Pooh is a fictional teddy bear created by English author A. A. Milne.
The first collection of stories about the character was the book Winnie-the-Pooh (1926),
and this was followed by The House at Pooh Corner (1928).
Milne also included a poem about the bear in the children's verse book When We Were Very Young (1924)
and many more in Now We Are Six (1927). All four volumes were illustrated by E. H. Shepard.
The Pooh stories have been translated into many languages, including Latin,
under the title Winnie ille Pu. These stories are about friendship, adventure, and simple joys."""

k_exp = 7  # k-gram size for experiment
w_exp = 6  # window size for experiment

change_percentages = np.linspace(0, 0.5, 11) # 0% to 50% change in 11 steps

results_replace = []
results_insert = []
results_delete = []

# 预计算原始文本的指纹
original_fingerprints = winnowing(base_text_variation, k_exp, w_exp)
if not original_fingerprints:
    print("Warning: Original text generated no fingerprints. Check k, w, and text length/content.")

print(f"Base text (len {len(base_text_variation)} chars), k={k_exp}, w={w_exp}")
print(f"Number of fingerprints for base text: {len(original_fingerprints)}")

for p in change_percentages:
    # 替换
    modified_text_replace = modify_text(base_text_variation, p, "replace")
    modified_fp_replace = winnowing(modified_text_replace, k_exp, w_exp)
    sim_replace = compare_documents_winnowing(original_fingerprints, modified_fp_replace)
    results_replace.append(sim_replace)
    print(f"  Replace {p*100:.1f}%: Sim={sim_replace:.4f}, ModFP#={len(modified_fp_replace)}")

    # 插入
    modified_text_insert = modify_text(base_text_variation, p, "insert")
    modified_fp_insert = winnowing(modified_text_insert, k_exp, w_exp)
    sim_insert = compare_documents_winnowing(original_fingerprints, modified_fp_insert)
    results_insert.append(sim_insert)
    print(f"  Insert  {p*100:.1f}%: Sim={sim_insert:.4f}, ModFP#={len(modified_fp_insert)}")

    # 删除
    modified_text_delete = modify_text(base_text_variation, p, "delete")
    modified_fp_delete = winnowing(modified_text_delete, k_exp, w_exp)
    sim_delete = compare_documents_winnowing(original_fingerprints, modified_fp_delete)
    results_delete.append(sim_delete)
    print(f"  Delete  {p*100:.1f}%: Sim={sim_delete:.4f}, ModFP#={len(modified_fp_delete)}")


# 绘图
plt.figure(figsize=(12, 7))
plt.plot(change_percentages * 100, results_replace, marker='o', label='字符替换 (Character Replace)')
plt.plot(change_percentages * 100, results_insert, marker='s', label='字符插入 (Character Insert)')
plt.plot(change_percentages * 100, results_delete, marker='^', label='字符删除 (Character Delete)')

plt.title(f'Winnowing 相似度 vs. 文本变化百分比 (k={k_exp}, w={w_exp})')
plt.xlabel('文本变化百分比 (%)')
plt.ylabel('Jaccard 相似度')
plt.legend()
plt.grid(True)
plt.ylim(0, 1.1) # Jaccard index is between 0 and 1
plt.xticks(change_percentages * 100)
plt.show()


## 5. 性能考量 (可选)

- **哈希函数选择**：MD5 或 SHA1 是常用的，但对于非常大规模的应用，更快的非加密哈希函数（如 MurmurHash, xxHash）如果能保证良好的分布性，也可以考虑。我们实现的 `hash_k_gram` 可以扩展。
- **k 和 w 的选择**：如前所述，它们直接影响指纹数量和算法的敏感性/特异性。
- **实现优化**：
    - 对于非常大的文档，可以流式处理 k-gram 和哈希值，而不是一次性全部加载到内存中。
    - 维护窗口中的最小哈希值可以使用更高效的数据结构（例如双端队列），但这会增加实现的复杂性。对于大多数情况，简单的滑动窗口扫描是可接受的。

### 比较 Winnowing, SimHash, MinHash (概念上)

| 特性         | Winnowing                                  | SimHash                                       | MinHash (用于集合相似性)                     |
|--------------|--------------------------------------------|-----------------------------------------------|----------------------------------------------|
| **基本单位**   | k-grams                                    | 文档特征 (如词语，加权)                         | 集合中的元素 (例如 shingles/k-grams)         |
| **指纹**     | k-gram 哈希值的子集 (窗口中的最小哈希)       | 单个固定长度的哈希值 (通常64或128位)            | 多个哈希值 (每个哈希函数对应一个最小值)        |
| **相似度计算** | Jaccard (基于选定的指纹哈希集合)           | Hamming 距离 (比较两个 SimHash 指纹)          | Jaccard (基于 MinHash 签名估算)              |
| **主要用途**   | 检测文档中的重复片段，抄袭检测             | 整体文档相似性，近似重复检测                  | 集合相似性估算，大规模聚类/去重              |
| **对局部性**  | 非常敏感，能定位相似片段                   | 对整体内容敏感，不直接定位片段                | 对元素存在性敏感，间接反映局部性（通过shingles）|
| **指纹数量**   | 可变，取决于 w 和文本内容                  | 固定 (1个)                                    | 固定 (等于哈希函数数量)                      |
| **鲁棒性**    | 对小的、局部的变化相对鲁棒                 | 对整体词频分布的变化鲁棒                      | 对元素增删有一定鲁棒性                       |

Winnowing 的一个关键优势是它能保证：如果两个文档共享一个长度至少为 `t` 的完全相同的子串，那么它们至少会共享一个指纹 (其中 `t = k + w - 1`)。这个特性称为“保证阈值”。

## 6. 总结与展望

Winnowing 是一种有效的文档指纹算法，特别适用于检测精确或近似的文本复制。通过调整 k 和 w 参数，可以平衡检测的敏感度和效率。

未来的工作可以包括：
- 探索不同哈希函数对性能和冲突率的影响。
- 将 Winnowing 与其他文本相似性技术（如 TF-IDF、词嵌入）结合，以获得更全面的文档理解。
- 针对特定类型的文档（如代码、结构化文本）优化预处理和 k-gram 生成步骤。
- 实现更高级的指纹存储和索引结构，以支持大规模文档集合的快速比较。